In [1]:
import numpy as np
from numpy import array as arr
from tabulate import tabulate

In [2]:
import sys
import socket
if socket.gethostname() == 'Birks-iMac.fritz.box':
    sys.path.append('/Users/Birk/Documents/Programmieren/Python/Eigener Code')
else:
    sys.path.append('/Home/ab227/linux/Eigener Code')
from pap import summen_fehler
from pap import _rundung, _rundung_einzel, _negativ_wird_null

# pap.vergleichstabelle()-Entwicklung

In [3]:
def _größenordnung(zahlen, art = int):
    if art == int:
        art = 'int64'
    elif art == float:
        art = 'float64'
    
    einsen = np.ones(np.shape(zahlen))   # 1 hat Größenordnung 0 (welches auch 0 hier haben soll).
    zahlen_kompatibel = np.where(zahlen == 0, einsen, zahlen)
    return arr(np.floor(np.log10(np.abs(zahlen_kompatibel))), dtype = art)

In [4]:
def _erste_ziffer(zahlen, art = float):
    größenordnungen = _größenordnung(zahlen, art = float)
    ziffern = np.abs(zahlen / 10**größenordnungen)
    if art == float:
        return ziffern
    if art == int:
        return np.int_(ziffern)
    

In [5]:
# Test: _erste_ziffer()

print(_erste_ziffer(arr([1.293, 37, 0.9304, -1.60, -0.999999, 0.4, 0.6, -0.4, -0.6])))     
print(_erste_ziffer(arr([1.293, 37, 0.9304, -1.60, -0.999999, 0.4, 0.6, -0.4, -0.6]), art = int))

[1.293   3.7     9.304   1.6     9.99999 4.      6.      4.      6.     ]
[1 3 9 1 9 4 5 4 5]


In [6]:
def _nachkommastelle(zahlen, sig_stellen = 1, sig_grenze = 1.0):
    
    über_der_grenze = _erste_ziffer(zahlen) >= sig_grenze
    sig_stellen_normal = np.full(np.shape(zahlen), sig_stellen)
    signifikante_stellen = np.where(über_der_grenze, sig_stellen_normal, sig_stellen_normal + 1)
    größenordnungen = _größenordnung(zahlen, art = float)
    
    nachkommastellen = -größenordnungen + signifikante_stellen - 1
    
    
    return nachkommastellen

In [7]:
# Test: _nachkommastelle(), _größenordnung()

zahlliste = arr([1e100, 3849184e38, -1.3984820e5, 48024.304, -40000, -39999.0, 1023, 999.9999, 483, -395.00, 
                 23.30489, 1.3848, 4.09, 3.688, -0.839, 0.094, 0.0003848, -0.00004848, 5.4839e-5, -3.9485e-7, 
                 7.38e-99])

nachkommastellen = _nachkommastelle(zahlliste, sig_grenze = 4.0)

print(_größenordnung(zahlliste))
print(nachkommastellen)
zahlliste_gerundet = _rundung(zahlliste, np.int_(nachkommastellen))
print(zahlliste_gerundet)

[100  44   5   4   4   4   3   2   2   2   1   0   0   0  -1  -2  -4  -5
  -5  -7 -99]
[-99. -43.  -4.  -4.  -4.  -3.  -2.  -2.  -2.  -1.   0.   1.   0.   1.
   1.   2.   5.   5.   5.   8.  99.]
[ 1.0e+100  3.8e+044 -1.4e+005  5.0e+004 -4.0e+004 -4.0e+004  1.0e+003
  1.0e+003  5.0e+002 -4.0e+002  2.3e+001  1.4e+000  4.0e+000  3.7e+000
 -8.0e-001  9.0e-002  3.8e-004 -5.0e-005  5.0e-005 -3.9e-007  7.0e-099]


In [10]:
def _füllen(string, menge = 1, index = 0, füllzeichen = ' '):
    string_neu = string[:index] + füllzeichen * menge + string[index:]
    return string_neu

In [12]:
# Test: _füllen()

_füllen('hi', index = len('hi'))

'hi '

In [14]:
def _zahlen_ausrichtung(zahlen_liste):
    länge_abs = len(zahlen_liste)
    anzahl = 7   #% (-,vk,.,nk,e,e-,ne, ende)

    test_minus   = ['-' in string[0] for string in zahlen_liste]
    test_komma   = ['.' in string for string in zahlen_liste]
    test_e       = ['e' in string for string in zahlen_liste]
    test_e_minus = ['e-' in string for string in zahlen_liste]
    längen_strings = [len(string) for string in zahlen_liste]
    indices = np.int_(np.zeros((länge_abs, anzahl + 1)))
    

    for i in range(länge_abs):
        indices[i][7] = längen_strings[i]
        if test_e[i]:
            indices[i][4] = zahlen_liste[i].find('e')
            if test_e_minus[i]:
                indices[i][5] = zahlen_liste[i].find('e-') + 1
                indices[i][6] = indices[i][5] + 1
            else: 
                indices[i][5:7] = [indices[i][4] + 1] * 2
        else:
            indices[i][4:7] = [längen_strings[i]] * 3
        if test_komma[i]:
            indices[i][2] = zahlen_liste[i].find('.')
            indices[i][3] = indices[i][2] + 1
        else:
            indices[i][2:4] = [indices[i][4]] * 2
        if test_minus[i]:
            indices[i][1] = 1
        else:
            indices[i][1] = 0

    längen = (indices.T[1:] - indices.T[:-1]).T
    längen_max = np.max(längen, axis = 0)
    längen_diff = längen_max[np.newaxis:] - längen

    längen_auswahl = [6, 5, 4, 3, 2, 1, 0]
    indices_auswahl = [5, 5, 4, 4, 2, 0, 0]   #% auf jeden Fall kommentieren.
    for i in range(länge_abs):
        for j in range(len(längen_auswahl)):
            zahlen_liste[i] = _füllen(zahlen_liste[i], menge = längen_diff[i][längen_auswahl[j]], 
                                    index = indices[i][indices_auswahl[j]])
    
    test_leerzeichen = [' ' in string[0] for string in zahlen_liste]
    if np.all(test_leerzeichen):
        for i in range(länge_abs):
            zahlen_liste[i] = zahlen_liste[i][1:]
    
    return zahlen_liste

In [15]:
def _listen_transponieren(listen_matrix):
    transponierte_matrix = []
    for i in range(len(listen_matrix[0])):
        zeile = []
        for j in range(len(listen_matrix)):
            zeile.append(listen_matrix[j][i])
        transponierte_matrix.append(zeile)
    return transponierte_matrix

In [18]:
# Test: _listen_transponieren()

test_listen_matrix = [['1', '2'], ['3', '4'], ['5', '6']]
test_listen_matrix = _listen_transponieren(test_listen_matrix)

In [19]:
def _zentrieren_rechts(string, länge_gewünscht, füllzeichen = ' '):
    länge_string = len(string)
    if länge_gewünscht <= länge_string:
        return string
    else:
        füll_länge        = länge_gewünscht - länge_string
        füll_länge_links  = int(np.ceil(füll_länge / 2))
        füll_länge_rechts = int(np.floor(füll_länge / 2))
        string_zentriert  = füllzeichen * füll_länge_links + string + füllzeichen * füll_länge_rechts
        return string_zentriert

In [20]:
def vergleichstabelle(werte:np.ndarray, einheit = '', faktor = 1, ausrichtung = 'blöcke'):
    # Vorbearbeitung der Werte
    if np.shape(werte) == (4,):   
        werte = np.array(werte, ndmin = 2).T
    werte = werte * faktor   # Umrechnung auf gewünschte Größenordnung bzw. Einheit
    werte[1::2] = np.abs(werte[1::2])      # Keine negativen Fehlerwerte
    ex, ex_fehler, theo, theo_fehler = werte
    anzahl_vergleiche = np.shape(werte)[1]
    anzahl_zahlen = 8    # Anzahl Zahlen, die in einem Vergleich vorkommen
    
    
    # Ausrechnen der Abweichungen
    abweichung_abs = ex - theo
    abweichung_abs_fehler = summen_fehler(arr([theo_fehler, ex_fehler]))
    abweichung_rel = abweichung_abs / theo * 100   # [%]
    abweichung_sig = np.abs(abweichung_abs / abweichung_abs_fehler)
    
    
    # Bestimmung der Nachkommastellen
    werte_absolut = arr([ex, theo, abweichung_abs])
    werte_fehler  = arr([ex_fehler, theo_fehler, abweichung_abs_fehler])
    werte_relativ = arr([abweichung_rel, abweichung_sig])
    
    nachkommastellen_fehler = _nachkommastelle(werte_fehler, sig_grenze = 3.95)
    nachkommastellen_absolut = nachkommastellen_fehler
    nachkommastellen_relativ = _nachkommastelle(werte_relativ, sig_stellen = 2, sig_grenze = 1.0)
    
    nachkommastellen = []
    for i in range(3):
        nachkommastellen.append(nachkommastellen_absolut[i])
        nachkommastellen.append(nachkommastellen_fehler[i])
    nachkommastellen.extend(nachkommastellen_relativ)
    nachkommastellen = np.array(nachkommastellen, dtype = int)
    
    
    # Rundung aller Werte
    alle_werte = arr([ex, ex_fehler, theo, theo_fehler, abweichung_abs, abweichung_abs_fehler,
                      abweichung_rel, abweichung_sig])
    alle_werte_gerundet = _rundung(alle_werte, nachkommastellen)
    größenordnungen = _größenordnung(alle_werte_gerundet)
    
    
    # Darstellung der einzelnen Werte
    alle_werte_strings = []
    for i in range(anzahl_zahlen):
        zahlen_strings = []
        for j in range(anzahl_vergleiche):
            if np.abs(größenordnungen[i][j]) < 5:
                nachkommastelle = _negativ_wird_null(nachkommastellen[i][j])
                wert_string = '{:.{prec}f}'.format(alle_werte_gerundet[i][j], prec = nachkommastelle)
            else:
                größenordnung = größenordnungen[i][j]
                wert_gestutzt = alle_werte_gerundet[i][j] * 10**(-float(größenordnung))
                präzision = nachkommastellen[i][j] + größenordnung

                wert_string = '{:.{prec}f}e{}'.format(wert_gestutzt, größenordnung, prec = präzision)
            zahlen_strings.append(wert_string)
        alle_werte_strings.append(zahlen_strings)
    
    
    # Formatierung der Werte zu einem schönen, gut lesbaren Vergleich
    if ausrichtung == 'blöcke':
        alle_werte_strings = arr(alle_werte_strings).T
        alle_vergleiche = []
        for i in range(anzahl_vergleiche):
            strings_einzelwerte = alle_werte_strings[i]

            # Anpassen der Absolutwerte aneinander
            strings_absolut = list(strings_einzelwerte[[0, 2, 4]])
            strings_absolut = _zahlen_ausrichtung(strings_absolut)

            # Anpassen der Fehlerwerte aneinander
            strings_fehler = list(strings_einzelwerte[[1, 3, 5]])
            strings_fehler = _zahlen_ausrichtung(strings_fehler)

            # Zusammenstellen des Vergleichs
            strings_plusminus = [strings_absolut[i] + ' ± ' + strings_fehler[i] 
                                 for i in range(len(strings_absolut))]
            string_rel, string_sig = [strings_einzelwerte[6] + ' %', strings_einzelwerte[7] + ' σ']
            if string_rel[0] == '-':
                string_sig = _füllen(string_sig)
            strings_vergleich = [*strings_plusminus, string_rel, string_sig]
            länge_gesamtstring = max([len(string) for string in strings_vergleich])
            for i in range(len(strings_vergleich)):
                strings_vergleich[i] = _zentrieren_rechts(strings_vergleich[i], länge_gesamtstring)
            alle_vergleiche.append(strings_vergleich)
            
            
    elif ausrichtung == 'liste':
        alle_zahlen = []
        for i in range(anzahl_zahlen):
            strings_einzelwerte = alle_werte_strings[i]
            strings_einzelwerte = _zahlen_ausrichtung(strings_einzelwerte)
            alle_zahlen.append(strings_einzelwerte)            
        strings_exp = [alle_zahlen[0][i] + ' ± ' + alle_zahlen[1][i] for i in range(anzahl_vergleiche)]
        strings_the = [alle_zahlen[2][i] + ' ± ' + alle_zahlen[3][i] for i in range(anzahl_vergleiche)]
        strings_abw = [alle_zahlen[4][i] + ' ± ' + alle_zahlen[5][i] for i in range(anzahl_vergleiche)]
        strings_rel = [alle_zahlen[6][i] + ' %' for i in range(anzahl_vergleiche)]
        strings_sig = [alle_zahlen[7][i] + ' σ' for i in range(anzahl_vergleiche)]
        alle_vergleiche = [strings_exp, strings_the, strings_abw, strings_rel, strings_sig]
    
    else:
        print(f'ausrichtung = "{ausrichtung}" ist kein gültiges Argument.')
        return
        
    
    # Erstellen der fertigen Tabellenstrings
    alle_vergleiche = _listen_transponieren(alle_vergleiche)
    for i in range(len(alle_vergleiche)):
        alle_vergleiche[i] = ' | '.join(alle_vergleiche[i])
            
    
    print('-' * len(alle_vergleiche[0]))
    for zeile in alle_vergleiche:
        print(zeile)  
    print('')

 ## Tests

In [21]:
# Test: Darstellung von Werten verschiedenster Größenordnungen

zahlliste = arr([1e100, 3849184e38, -1.3984820e5, 48024.304, -40000, -39999.0, 1023, 483, -395.00, 23.30489, 
                 4.09, 3.688, 1.3848, -0.839, 0.094, 0.0003848, -0.00004848, 5.4839e-5, -3.9485e-7, 7.38e-99])
fehlerliste = zahlliste / 77
theoliste = zahlliste * 1.05

# Modifikation um Darstellung von negativen Zahlen weiter zu testen
theoliste[0] = theoliste[0] * 0.9    # Test "kein -"
theoliste[-4] = theoliste[-4] * 0.9  # Test "alle -"

alle_werte = arr([zahlliste, fehlerliste, theoliste, fehlerliste]).T


for i in range(len(zahlliste)):
    vergleichstabelle(alle_werte[i])

------------------
1.000e100 ± 1.3e98
9.45 e 99 ± 1.3e98
5.5  e 98 ± 1.8e98
       5.8 %      
       3.0 σ      

---------------
 3.85e44 ± 5e42
 4.04e44 ± 5e42
-1.9 e43 ± 7e42
     -4.8 %    
      2.7 σ    

-----------------
  -1.398e5 ± 1800
  -1.468e5 ± 1800
7000       ± 2600
      -4.8 %     
       2.7 σ     

-----------
48000 ± 600
50400 ± 600
-2400 ± 900
   -4.8 %  
    2.7 σ  

------------
-40000 ± 500
-42000 ± 500
  2000 ± 700
   -4.8 %   
    2.7 σ   

------------
-40000 ± 500
-42000 ± 500
  2000 ± 700
   -4.8 %   
    2.7 σ   

---------
1023 ± 13
1074 ± 13
 -51 ± 19
  -4.8 % 
   2.7 σ 

-------
483 ± 6
507 ± 6
-24 ± 9
 -4.8 %
  2.7 σ

--------
-395 ± 5
-415 ± 5
  20 ± 7
 -4.8 % 
  2.7 σ 

------------
23.30 ± 0.30
24.47 ± 0.30
-1.2  ± 0.4 
   -4.8 %   
    2.7 σ   

------------
 4.09 ± 0.05
 4.29 ± 0.05
-0.20 ± 0.08
   -4.8 %   
    2.7 σ   

------------
 3.69 ± 0.05
 3.87 ± 0.05
-0.18 ± 0.07
   -4.8 %   
    2.7 σ   

--------------
 1.385 ± 0.018
 1.454 ± 0.018
-

In [22]:
# Test: Listendarstellung der Vergleiche mit vielen Größenordnungen

# Um den Output breit genug für die Tabelle zu machen.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [23]:
vergleichstabelle(alle_werte.T, ausrichtung = 'liste')

----------------------------------------------------------------------------------------------------------------------------
     1.000   e 100 ±    1.3   e  98 |      9.45    e 99 ±    1.3   e  98 |     5.5   e  98 ±    1.8   e  98 |  5.8 % | 3.0 σ
     3.85    e  44 ±    5     e  42 |      4.04    e 44 ±    5     e  42 |    -1.9   e  43 ±    7     e  42 | -4.8 % | 2.7 σ
    -1.398   e   5 ± 1800           |     -1.468   e  5 ± 1800           |  7000           ± 2600           | -4.8 % | 2.7 σ
 48000             ±  600           |  50400            ±  600           | -2400           ±  900           | -4.8 % | 2.7 σ
-40000             ±  500           | -42000            ±  500           |  2000           ±  700           | -4.8 % | 2.7 σ
-40000             ±  500           | -42000            ±  500           |  2000           ±  700           | -4.8 % | 2.7 σ
  1023             ±   13           |   1074            ±   13           |   -51           ±   19           | -4.8 % | 2.7 σ


In [24]:
# Test: Listendarstellungen bei kleineren Listen und ähnlicheren Werten.

vergleichstabelle(alle_werte[7:10].T, ausrichtung = 'liste')
vergleichstabelle(alle_werte[0:2].T, ausrichtung = 'liste')
vergleichstabelle(alle_werte[2:5].T, ausrichtung = 'liste')

--------------------------------------------------------------
 483    ± 6    |  507    ± 6    | -24   ± 9   | -4.8 % | 2.7 σ
-395    ± 5    | -415    ± 5    |  20   ± 7   | -4.8 % | 2.7 σ
  23.30 ± 0.30 |   24.47 ± 0.30 |  -1.2 ± 0.4 | -4.8 % | 2.7 σ

-------------------------------------------------------------------------
1.000e100 ± 1.3e98 | 9.45e99 ± 1.3e98 |  5.5e98 ± 1.8e98 |  5.8 % | 3.0 σ
3.85 e 44 ± 5  e42 | 4.04e44 ± 5  e42 | -1.9e43 ± 7  e42 | -4.8 % | 2.7 σ

-------------------------------------------------------------------------
    -1.398e5 ± 1800 |     -1.468e5 ± 1800 |  7000 ± 2600 | -4.8 % | 2.7 σ
 48000       ±  600 |  50400       ±  600 | -2400 ±  900 | -4.8 % | 2.7 σ
-40000       ±  500 | -42000       ±  500 |  2000 ±  700 | -4.8 % | 2.7 σ



In [25]:
# Test: Vergleich der Block-Darstellung mit der Listendarstellung

zahlliste = arr([39950, -0.000039999, -0.0000398384, -0.0000393884])
fehlerliste = zahlliste / 10
theoliste = zahlliste * 1.01
alle_werte = arr([zahlliste, fehlerliste, theoliste, fehlerliste])

vergleichstabelle(alle_werte, ausrichtung = 'blöcke')
vergleichstabelle(alle_werte, ausrichtung = 'liste')

--------------------------------------------------------------------------------
40000 ± 4000 | -4.0     e-5 ± 4e-6 | -4.0     e-5 ± 4e-6 | -3.94    e-5 ± 3.9e-6
40000 ± 4000 | -4.0     e-5 ± 4e-6 | -4.0     e-5 ± 4e-6 | -3.98    e-5 ± 3.9e-6
   -0 ± 6000 |  0.000000    ± 6e-6 |  0.000000    ± 6e-6 |  0.000000    ± 6  e-6
   -0.99 %   |       -0.99 %       |       -0.99 %       |        -0.99 %       
   0.071 σ   |        0.071 σ      |        0.071 σ      |         0.071 σ      

-------------------------------------------------------------------------------------------
40000       ± 4000      | 40000       ± 4000      | -0        ± 6000    | -0.99 % | 0.071 σ
   -4.0 e-5 ±    4  e-6 |    -4.0 e-5 ±    4  e-6 |  0.000000 ±    6e-6 | -0.99 % | 0.071 σ
   -4.0 e-5 ±    4  e-6 |    -4.0 e-5 ±    4  e-6 |  0.000000 ±    6e-6 | -0.99 % | 0.071 σ
   -3.94e-5 ±    3.9e-6 |    -3.98e-5 ±    3.9e-6 |  0.000000 ±    6e-6 | -0.99 % | 0.071 σ



In [26]:
# Test: Falsches ausrichtung-Argument
vergleichstabelle(alle_werte[7:10].T, ausrichtung = 'listen')

ausrichtung = "listen" ist kein gültiges Argument.


In [27]:
# Test der Funktion:

# Test mit normalen Werten
vergleichstabelle(arr([0.123, 0.012, 1.098, 0.0987]))

# Tests mit negativen Werten und negativen Fehlern
vergleichstabelle(arr([ 0.123, -0.012,  1.098, 0.0987]))
vergleichstabelle(arr([-0.123,  0.012, -1.098, 0.0987]))
vergleichstabelle(arr([ 0.123,  0.012, -1.098, 0.0987]))


# Test mit zwei Vergleichen
vergleichstabelle(arr([[0.123, 400 / 7], 
                       [0.012, 23 / 7], 
                       [1.098, 337 / 7], 
                       [0.987, 29 / 7]]))

# Test der Rundung (auch bei rel. und sig. Abweichung)
vergleichstabelle(arr([[284.3858, 0.07538, 0.00829339400, -0.123], 
                       [40.0000 , 0.00400, 0.00003950000,  0.0012], 
                       [317.3048, 0.07536, 0.00999999900, -1.098], 
                       [39.9999 , 0.00399, 0.00003949990,  0.0000098]]))

# Test mit längeren relativen als ±-Strings
vergleichstabelle(arr([1e-7, 4,  1e-5, 4]))

--------------
 0.123 ± 0.012
 1.10  ± 0.10 
-0.98  ± 0.10 
     -89 %    
     9.8 σ    

--------------
 0.123 ± 0.012
 1.10  ± 0.10 
-0.98  ± 0.10 
     -89 %    
     9.8 σ    

--------------
-0.123 ± 0.012
-1.10  ± 0.10 
 0.98  ± 0.10 
     -89 %    
     9.8 σ    

--------------
 0.123 ± 0.012
-1.10  ± 0.10 
 1.22  ± 0.10 
    -110 %    
      12 σ    

---------------------------
 0.123 ± 0.012 | 57.1 ± 3.3
 1.1   ± 1.0   | 48   ± 4  
-1.0   ± 1.0   |  9   ± 5  
     -89 %     |    19 %   
     0.99 σ    |    1.7 σ  

---------------------------------------------------------------------
280 ± 40 | 0.075 ± 0.004 |  0.008293 ± 4.0e-5 | -0.1230   ± 0.0012   
320 ± 40 | 0.075 ± 0.004 |  0.010000 ± 3.9e-5 | -1.098000 ± 1.0   e-5
-30 ± 60 | 0.000 ± 0.006 | -0.00171  ± 6  e-5 |  0.9750   ± 0.0012   
  -10 %  |    0.027 %    |        -17 %       |         -89 %        
  0.58 σ |    0.0035 σ   |         31 σ       |          810 σ       

---------
   0 ± 4 
   0 ± 4 
  -0 ± 6 
  -99 

### Tests mit richtigen Werten aus PAP-Experimenten

In [28]:
# Test: standard Wertdarstellung
werte = arr([75.31709457,  0.76723428, 66.,          0.5       ])
vergleichstabelle(werte)


# Zum Vergleich: Resultat mit der ursprünglischen Funktion
'''
Tabelle 5: Literaturvergleich Inklinationswinkel

Angaben in °
-------------------  ----
Messwert             75.3
dessen Fehler         0.8
Literaturwert        66.0
dessen Fehler         0.5
rel. Abweichung [%]  14.1
Sigma                10.2
''';

----------
75.3 ± 0.8
66.0 ± 0.5
 9.3 ± 0.9
   14 %   
   10 σ   



In [29]:
# Test: standard Wertdarstellung (bes. mit unterschiedlich vielen Nachkommastellen)
werte = arr([[1.25153244e+00, 1.38557443e+00, 1.40897994e+00],
             [2.39146223e-02, 1.56325105e-02, 2.27141900e-02],
             [1.40000000e+00, 1.40000000e+00, 1.64800000e+00],
             [1.00000000e-03, 1.00000000e-03, 5.00000000e-04]])
vergleichstabelle(werte)


# Zum Vergleich: Resultat mit der ursprünglischen Funktion
'''
Tabelle 1: Vergleich der Viskositäten nach Stokes bzw. Hagen-Poiseuille

Adiabatenkoeffizient      Clément-Desormes Luft    Rüchhardt Luft    Rüchhardt Argon
----------------------  -----------------------  ----------------  -----------------
exp. Wert                                  1.25              1.39               1.41
dessen Fehler                              0.02              0.02               0.02
the. Wert                                  1.40              1.40               1.65
dessen Fehler                              0.00              0.00               0.00
rel. Abweichung [%]                      -10.60             -1.03             -14.50
Sigma                                      6.20              0.92              10.52
''';

------------------------------------------------------
 1.252  ± 0.024  |  1.386  ± 0.016  |  1.409  ± 0.023 
 1.4000 ± 0.0010 |  1.4000 ± 0.0010 |  1.6480 ± 0.0005
-0.148  ± 0.024  | -0.014  ± 0.016  | -0.239  ± 0.023 
      -11 %      |      -1.0 %      |       -15 %     
      6.2 σ      |       0.92 σ     |        11 σ     



In [30]:
# Test: theoretischer Wert ist exakt & faktor-Argument
werte = arr([1.39515138e-23, 2.90095704e-25, 1.38064900e-23, 0.00000000e+00])
vergleichstabelle(werte, faktor = 1e23)


# Zum Vergleich: Resultat mit der ursprünglischen Funktion
'''
Tabelle 1: Literaturvergleich Boltzmannkonstante

Angaben in 10^(-23) J/K
-------------------------  ----
Messwert                   1.40
dessen Fehler              0.03
Literaturwert              1.38
dessen Fehler              0.00
rel. Abweichung [%]        1.05
Sigma                      0.50
''';

-------------
1.395 ± 0.029
1.4   ± 0.0  
0.015 ± 0.029
    1.1 %    
    0.50 σ   



In [31]:
# Test: Listenansicht und theoretischer Wert ist 0
werte = arr([[-0.0631769, 1.41696751, 2.48194946, 3.4566787, 4.55776173],
             [0.05460126, 0.05472070, 0.05496739, 0.0553094, 0.05582687],
             [0,          1.43030,    2.45902,    3.47089,   4.47741   ],
             [0,          0,          0,          0,         0         ]])
vergleichstabelle(werte)


# Zum Vergleich: Resultat mit der ursprünglischen Funktion
'''
Vergleich der Ordnungszahlen der Maxima

  theoretisch    experimentell    Abweichung [%]    Sigma
-------------  ---------------  ----------------  -------
         0.00   -0.06 +/- 0.05            -inf        1.2
         1.43    1.42 +/- 0.05              -0.9      0.2
         2.46    2.48 +/- 0.05               0.9      0.4
         3.47    3.46 +/- 0.06              -0.4      0.3
         4.48    4.56 +/- 0.06               1.8      1.4
         
<ipython-input-7-ebcfbd735655>:12: RuntimeWarning: divide by zero encountered in true_divide
  abweichung_rel = abweichung / liste1 * 100   # [%], relative Abweichung
''';

<ipython-input-20-1f65b9a2a0e9>:15: RuntimeWarning: divide by zero encountered in true_divide
  abweichung_rel = abweichung_abs / theo * 100   # [%]
<ipython-input-4-1478c7706986>:3: RuntimeWarning: invalid value encountered in true_divide
  ziffern = np.abs(zahlen / 10**größenordnungen)


OverflowError: signed integer is less than minimum

In [32]:
# Test: fehler_rel == fehler_sig == 0
werte = arr([[12368.47501413,  7421.08500848],
             [ 1124.64056495,   836.7201341 ],
             [12368.47501413,  7421.08500848],
             [  623.31254737,   620.18815701]])
vergleichstabelle(werte)


# Zum Vergleich: Resultat mit der ursprünglischen Funktion
'''
Vergleich der Wellenzahl 2

  [mm^-1]       theoretisch     experimentell    Abweichung [%]    Sigma
---------  ----------------  ----------------  ----------------  -------
       a)  12.368 +/- 1.125  12.368 +/- 0.623              -0.0      0.0
       b)   7.421 +/- 0.837   7.421 +/- 0.620               0.0      0.0
''';

--------------------------
12400 ± 1100 | 7400 ±  800
12400 ±  600 | 7400 ±  600
    0 ± 1300 |    0 ± 1000
   0.00 %    |    0.00 %  
   0.00 σ    |    0.00 σ  



In [33]:
# Test: Faktorargument; experimenteller und Theoretischer wert sollen andere Namen haben
werte = arr([3580.,  100., 3080.,   90.])
vergleichstabelle(werte, faktor = 1e-3)


# Zum Vergleich: Resultat mit der ursprünglischen Funktion
'''
Tabelle 2: Vergleich der Grenzfrequenzen aus der Hochpass- und Tiefpassbestimmung

Grenzfrequenz      [kHz]
---------------  -------
Hochpass            3.58
dessen Fehler       0.10
Tiefpass            3.08
dessen Fehler       0.09
Abweichung          0.50
Sigma               3.72
''';

-----------
3.58 ± 0.10
3.08 ± 0.09
0.50 ± 0.13
    16 %   
   3.7 σ   



In [34]:
# Test: Vergleiche, in denen Werte fehlen
werte = arr([[820.5, 571.4, 501.0, 470.2, 454.3, None, 433.6, 430.7, None, 423.5],
             [0.8493218002880191, 1.1890505204032267, 0.5945252602016133,
              1.2739827004320285, 1.6986436005760381, None, 2.1233045007200477,
              2.1233045007200477, None, 1.2739827004320285],
             [820.5, 570.7094072095474, 500.2226094489081, 468.7724747247101,
              451.650407609065, 441.1913744753898, 434.29622148706073, 429.4949134280389,
              426.0102653247866, 423.39753086740905],
             [0.8493218002880192, 0.4109085361760692, 0.3156761241481166,
              0.2772294138792619, 0.2573474764388973, 0.24556650385495118,
              0.23795081828704262, 0.23271862938516857, 0.2289576870796812,
              0.22615788961099104]])
vergleichstabelle(werte)


# Zum Vergleich: Resultat mit der ursprünglischen Funktion
'''
Linien der 1. Nebenserie [nm]

erwartet       gemessen         Abweichung    Sigma
-------------  -------------  ------------  -------
820.5 +/- 0.8  820.5 +/- 0.8           0.0      0.0
570.7 +/- 0.4  571.4 +/- 1.2           0.7      0.5
500.2 +/- 0.3  501.0 +/- 0.6           0.8      1.2
468.8 +/- 0.3  470.2 +/- 1.3           1.4      1.1
451.7 +/- 0.3  454.3 +/- 1.7           2.6      1.5
441.2 +/- 0.2
434.3 +/- 0.2  433.6 +/- 2.1          -0.7      0.3
429.5 +/- 0.2  430.7 +/- 2.1           1.2      0.6
426.0 +/- 0.2
423.4 +/- 0.2  423.5 +/- 1.3           0.1      0.1
''';

TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

## Skizzen zu Darstellungsvarianten